## lab2

In [3]:
#Install aws cli

pip3 install --upgrade awscli

     |################################| 1.6MB 14.3MB/s eta 0:00:01
     |################################| 71kB 11.3MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/36/fa/08e9e6e0e3cbd1d362c3bbee8d01d0aedb2155c4ac112b19ef3cae8eed8d/docutils-0.14-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/db/c8/7dcf9dbcb22429512708fe3a547f8b6101c0d02137acbd892505aee57adf/colorama-0.3.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9e/a3/1d13970c3f36777c583f136c136f804d70f500168edc1edea6daa7200769/PyYAML-3.13.tar.gz
     |################################| 51kB 7.1MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/cd/6d/7c1b99671d14db8e0318ceb3d5655eb29a34b25174fe5a71edb076ef81c8/botocore-1.12.165-py2.py3-none-any.whl
     |################################| 81kB 7.7MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/83/94/7179c3832a6d45b266ddb2aac329e101367fbdb11f425f13771d27f225bb/jm

In [4]:
aws --version

aws-cli/1.16.175 Python/3.6.7 Linux/4.15.0-1032-gcp botocore/1.12.165


In [18]:
# setup credentials
if [ ! -e /i3c/.secrets/.secrets/aws ]; then
mkdir -p /i3c/.secrets/.secrets/aws
/i sc aws/keyId
/i sc aws/accessKey
if [ ! -e /run/secrets ]; then
    mkdir -p /run/secrets
    ln -sf /i3c/.secrets/.secrets/aws /run/secrets/aws
    echo "<key id>" > /run/secrets/aws/keyId
    echo " <key secret>" > /run/secrets/aws/accessKey
fi
fi

#(put Your secrets into /run/secrets/[name])

mkdir -p $HOME/.aws

cat >$HOME/.aws/config <<EOF
[default]
aws_access_key_id = $(/r secret aws/keyId)
aws_secret_access_key = $(/r secret aws/accessKey)
region = eu-west-1
EOF

# secure the file
chmod 600 $HOME/.aws/config

#Optionally set an environment variable pointing to the config file, especially if you put it in a non-standard location. For future convenience, also add this line to your $HOME/.bashrc
export AWS_CONFIG_FILE=$HOME/.aws/config

Secret exists. Delete it first with /i sd name
Secret exists. Delete it first with /i sd name


In [21]:
#ls -al /run/secrets/aws/keyId 
#echo $(/r secret aws/keyId)
#/r secret aws/keyId
#cat /run/secrets/aws/keyId 

In [ ]:
aws ec2 create-key-pair --key-name ArchitechKeys

In [103]:
aws cloudformation create-stack --capabilities CAPABILITY_IAM --region eu-west-1 --stack-name ArchitechWA \
 --template-url https://tmpi3c.s3-eu-west-1.amazonaws.com/TestApp3.json \
 --parameters ParameterKey=KeyName,ParameterValue=ArchitechKeys  ParameterKey=DBUser,ParameterValue=user \
 ParameterKey=DBPassword,ParameterValue=user12345

{
    "StackId": "arn:aws:cloudformation:eu-west-1:688124483335:stack/ArchitechWA/154458c0-8d33-11e9-96da-0663408f5242"
}


In [106]:
#apt-get install -y jq

aws cloudformation describe-stacks --stack-name ArchitechWA | jq -r  '.Stacks[0].StackStatus'



CREATE_COMPLETE


In [107]:
cout=$(aws cloudformation describe-stacks --stack-name ArchitechWA | jq -r  '.Stacks[0].Outputs[0].OutputValue')
if [ "$cout" == "null" ]; then
    echo "Status:"
    aws cloudformation describe-stacks --stack-name ArchitechWA | jq -r  '.Stacks[0].StackStatus'
else
    echo "Application available at:"
    echo "http://"$cout
fi


Application available at:
http://Archi-LoadB-1VKJVILLX6273-2080392764.eu-west-1.elb.amazonaws.com


In [104]:
aws cloudformation describe-stacks --stack-name ArchitechWA

{
    "Stacks": [
        {
            "StackId": "arn:aws:cloudformation:eu-west-1:688124483335:stack/ArchitechWA/154458c0-8d33-11e9-96da-0663408f5242",
            "StackName": "ArchitechWA",
            "Description": "Simple Stack",
            "Parameters": [
                {
                    "ParameterKey": "KeyName",
                    "ParameterValue": "ArchitechKeys"
                },
                {
                    "ParameterKey": "DBPassword",
                    "ParameterValue": "****"
                },
                {
                    "ParameterKey": "DBAllocatedStorage",
                    "ParameterValue": "5"
                },
                {
                    "ParameterKey": "DBInstanceClass",
                    "ParameterValue": "db.t2.small"
                },
                {
                    "ParameterKey": "DBName",
                    "ParameterValue": "MyDatabase"
                },
                {
                    "ParameterK

In [110]:
#aws s3 cp --no-sign-request s3://tomash-public/examples/php/index.php index.php 
#wget https://raw.githubusercontent.com/i3c-cloud/tmp/master/index.php -O index.php

aws s3 cp s3://tmpi3c/TestApp3.json TestApp3.json

download: s3://tmpi3c/TestApp3.json to ./TestApp3.json            


In [111]:
#cat index.php
cat TestApp3.json          

{
    "AWSTemplateFormatVersion" : "2010-09-09",
  
    "Description" : "Simple Stack",
  
  
    "Parameters" : {
  
      "InstanceType" : {
        "Description" : "Server instance type",
        "Type" : "String",
        "Default" : "t2.small",
        "AllowedValues" : [ "t1.micro", "t2.nano", "t2.micro", "t2.small", "t2.medium", "t2.large", "m1.small", "m1.medium", "m1.large", "m1.xlarge", "m2.xlarge", "m2.2xlarge", "m2.4xlarge", "m3.medium", "m3.large", "m3.xlarge", "m3.2xlarge", "m4.large", "m4.xlarge", "m4.2xlarge", "m4.4xlarge", "m4.10xlarge", "c1.medium", "c1.xlarge", "c3.large", "c3.xlarge", "c3.2xlarge", "c3.4xlarge", "c3.8xlarge", "c4.large", "c4.xlarge", "c4.2xlarge", "c4.4xlarge", "c4.8xlarge", "g2.2xlarge", "g2.8xlarge", "r3.large", "r3.xlarge", "r3.2xlarge", "r3.4xlarge", "r3.8xlarge", "i2.xlarge", "i2.2xlarge", "i2.4xlarge", "i2.8xlarge", "d2.xlarge", "d2.2xlarge", "d2.4xlarge", "d2.8xlarge", "hi1.4xlarge", "hs1.8xlarge", "cr1.8xlarge", "cc2.8xlarge", "cg1.4xlarge"]

           "VpcId" : { "Ref" : "VPC" },
           "InternetGatewayId" : { "Ref" : "InternetGateway" }
         }
      },
  
      "PublicRouteTable" : {
        "Type" : "AWS::EC2::RouteTable",
        "Properties" : {
          "VpcId" : {"Ref" : "VPC"},
          "Tags" : [ {"Key" : "Name", "Value" : "PublicRoutTable" } ]
        }
      },
  
      "PrivateRouteTable" : {
        "Type" : "AWS::EC2::RouteTable",
        "Properties" : {
          "VpcId" : {"Ref" : "VPC"},
          "Tags" : [ {"Key" : "Name", "Value" : "PrivateRoutTable" } ]
        }
      },
  
      "PrivateRoute" : {
        "Type" : "AWS::EC2::Route",
        "DependsOn" : "AttachGateway",
        "Properties" : {
          "RouteTableId" : { "Ref" : "PrivateRouteTable" },
          "DestinationCidrBlock" : "0.0.0.0/0",
          "NatGatewayId" : { "Ref" : "NAT" }
        }
      },
  
      "PublicRoute" : {
        "Type" : "AWS::EC2::Route",
        "DependsOn" : "AttachGateway",
        "Properties" : {


           } ],
           "ComparisonOperator": "GreaterThanThreshold",
           "MetricName": "CPUUtilization"
        }
     },
     "CPUAlarmLow": {
        "Type": "AWS::CloudWatch::Alarm",
        "Properties": {
           "AlarmDescription": "Scale-down if CPU < 30% for 10 minutes",
           "MetricName": "CPUUtilization",
           "Namespace": "AWS/EC2",
           "Statistic": "Average",
           "Period": "300",
           "EvaluationPeriods": "2",
           "Threshold": "30",
           "AlarmActions": [ { "Ref": "WebServerScaleDownPolicy" } ],
           "Dimensions": [
             {
               "Name": "AutoScalingGroupName",
               "Value": { "Ref": "AutoScalingGroup" }
             }
           ],
           "ComparisonOperator": "LessThanThreshold"
         }
       },
  
      "EIP" : {
        "Type" : "AWS::EC2::EIP",
        "DependsOn" : "AttachGateway",
        "Properties" : {
          "Domain" : "vpc"
        }
      },
  
      "NAT" : {


In [102]:
#aws cloudformation delete-stack --stack-name ArchitechWA
aws cloudformation delete-stack --stack-name ArchitechWA5